In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from lxml import html

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}

In [60]:
def get_cash_dividend_payouts( stock_id, 
                               year_to_check=5):
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pd.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    cash_dividend_payouts = df["現金股利"][:year_to_check].astype('float')
    return list(cash_dividend_payouts)

In [61]:
def get_ROEs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/financial.aspx?no={}&t=3&st=2&q=3".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pd.read_html(data.prettify())

    df = dfs[0]
    ROEs = [float(ROE.strip('%')) for ROE in df["年度ROE"][:year_to_check]]
    return ROEs

In [62]:
def get_PEratios( stock_id,
                  year_to_check=20):
    
    res = requests.get("https://histock.tw/stock/{}/%E6%9C%AC%E7%9B%8A%E6%AF%94".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pd.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    PEratio_cols = [col for col in df.columns if '本益比' in col]
    PEratios = []
    for col in PEratio_cols:
        PEratios += list(df[col])

    return PEratios[:year_to_check]

In [63]:
def get_prev_net_val_per_share( stock_id, 
                                prev_year="2019"):

    res = requests.get("https://histock.tw/stock/{}/%E6%AF%8F%E8%82%A1%E6%B7%A8%E5%80%BC".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")
    dfs = pd.read_html(data.prettify())

    df = dfs[0]

    prev_net_val_per_share = df.iloc[3][prev_year]
    return prev_net_val_per_share

In [64]:
def get_EPSs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pd.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]

    EPSs = df["EPS"][:year_to_check].astype('float')
    return list(EPSs)

In [65]:
def getCompanyAttrs(stock_id):
    res = requests.get("https://histock.tw/stock/financial.aspx?no={}".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    tree = html.fromstring(res.text)
    company_name = tree.xpath('//*[@id="form1"]/div[4]/div[4]/div/div[1]/div[2]/div[1]/div/h3/a')[0].text

    cur_price = tree.xpath('//*[@id="CPHB1_Price1_lbTPrice"]/a/span')[0].text

    volume = tree.xpath('//*[@id="CPHB1_Price1_lbTVolume"]/span')[0].text
    attr = tree.xpath('//*[@id="CPHB1_Price1_lbStockClass"]')[0].text.split(' ')
    stock_type = attr[0]
    industry = attr[1]
    
    return company_name, cur_price, volume, stock_type, industry

In [69]:
def getBuyPrice(cur_dividend_payout, 
                sell_price, 
                ror, 
                year_to_hold,
                display_log=False):
    
    buy_price = 0
    for year in range(1, year_to_hold+1):
        if year == year_to_hold:
            buy_price += (cur_dividend_payout + sell_price) / ((1 + ror) ** year )
        else: 
            buy_price += cur_dividend_payout / ((1 + ror) ** year )
        
        if display_log:
            print ("year {} => buy_price: {:.2f}".format(year, buy_price))
    
    if display_log:
        print ("buy_price: {:.2f}\n".format(buy_price))
    
    return buy_price

In [77]:
def analyzeStock(stock_id,    
                 year_to_hold = 8,
                 prev_year="2019",
                 max_PEratio=12, 
                 verbose = 0 ):
                 
    cash_dividend_payouts = get_cash_dividend_payouts( stock_id, year_to_check=5 )
    EPSs = get_EPSs( stock_id, year_to_check=5 )
    ROEs = get_ROEs( stock_id, year_to_check=5 )
    PEratios = get_PEratios( stock_id, year_to_check=20 )
    prev_net_val_per_share = get_prev_net_val_per_share( stock_id, prev_year )
    dividend_payout_ratios = np.array(cash_dividend_payouts) / np.array(EPSs)
    
    exptect_ROE = np.mean(ROEs) / 100 
    exptect_dividend_payout_ratio = np.mean(dividend_payout_ratios) 

    
    prev_EPS = EPSs[-1]
    prev_dividend_payout = cash_dividend_payouts[-1]
    cur_net_val_per_share = prev_net_val_per_share + (prev_EPS - prev_dividend_payout)

    cur_EPS = exptect_ROE * cur_net_val_per_share

    cur_dividend_payout = cur_EPS * exptect_dividend_payout_ratio

    EPS_8th = exptect_ROE * prev_net_val_per_share

    expect_PEratio = np.mean(PEratios)
    expect_PEratio = min( expect_PEratio, max_PEratio )

    sell_price = EPS_8th * expect_PEratio
    
    if verbose >= 1:
        print ('{:>36}  {}'.format("stock id", stock_id ))
        print ('{:>36}  {:>4.2f} (year)'.format("exptect_ROE", exptect_ROE ))
        print ('{:>36}  {:>4.2f}'.format("exptect_dividend_payout_ratio", exptect_dividend_payout_ratio ))
        print ('{:>36}  {:>4.2f}'.format("cur_net_val_per_share (accumulated)", cur_net_val_per_share ))
        print ('{:>36}  {:>4.2f}'.format("cur_EPS", cur_EPS ))
        print ('{:>36}  {:>4.2f}'.format("cur_dividend_payout", cur_dividend_payout ))
        print ('{:>36}  {:>4.2f}'.format("EPS_8th", EPS_8th ))
        print ('{:>36}  {:>4.2f}'.format("expect_PEratio", expect_PEratio ))
        print ('{:>36}  {:>4.2f}\n'.format("sell_price", sell_price ))    
    
    
    buy_price_ror10 = getBuyPrice( cur_dividend_payout, sell_price, 0.10, year_to_hold, display_log=verbose>=2 )
    buy_price_ror15 = getBuyPrice( cur_dividend_payout, sell_price, 0.15, year_to_hold, display_log=verbose>=2 )

    company_name, cur_price, volume, stock_type, industry = getCompanyAttrs(stock_id)
    
    return buy_price_ror10, buy_price_ror15, sell_price, company_name, cur_price, volume, stock_type, industry

In [83]:
buy_prices_ror10 = []
buy_prices_ror15 = []
sell_prices = []
company_names = []
cur_prices = []
volumes = []
stock_types = []
industrys = []
stock_ids = [2904, 3567, 6216, 4426, 2633, 3169, 6257, 6210, 9926, 8424, 6532, 2441, 3556, 6173, 6525, 3030, 6523, 9943, 6263, 6147, 6202, 6452, 4549, 6588, 3526, 1301, 2114, 6411, 2455, 4205, 5347, 6224, 4163, 8446, 2428, 8416, 3217, 4119, 3532, 6569, 6568, 2707, 2458, 4107, 8422, 1707, 8437, 4137, 8083, 5287, 6803, 6582, 8341, 6146, 5474, 1565, 1256, 2059, 3105, 8464, 1476, 3152, 6488, 2330, 3529, 3293, 6409, 4966, 5274, 5269, 6415, 3008]

cnt = 0
for stock_id in stock_ids:
    
    try:
        buy_price_ror10, buy_price_ror15, sell_price, company_name, cur_price, volume, stock_type, industry = \
            analyzeStock(stock_id, verbose=0)

        buy_prices_ror10.append(buy_price_ror10)
        buy_prices_ror15.append(buy_price_ror15)
        sell_prices.append(sell_price)
        company_names.append(company_name)
        cur_prices.append(cur_price)
        volumes.append(volume)
        stock_types.append(stock_type)
        industrys.append(industry)
        
    except:
        print ("fail stock id:", stock_id)
    
    cnt += 1    
    if cnt % 10 == 0:
        print ("{} stocks have been analyzed.".format(cnt))

10 stocks have been analyzed.
20 stocks have been analyzed.
fail stock id: 6588
30 stocks have been analyzed.
40 stocks have been analyzed.
50 stocks have been analyzed.
fail stock id: 6582
60 stocks have been analyzed.
70 stocks have been analyzed.


In [105]:
stock_ids = [2904, 3567, 6216, 4426, 2633, 3169, 6257, 6210, 9926, 8424, 6532, 2441, 3556, 6173, 6525, 3030, 6523, 9943, 6263, 6147, 6202, 6452, 4549, 3526, 1301, 2114, 6411, 2455, 4205, 5347, 6224, 4163, 8446, 2428, 8416, 3217, 4119, 3532, 6569, 6568, 2707, 2458, 4107, 8422, 1707, 8437, 4137, 8083, 5287, 6803, 8341, 6146, 5474, 1565, 1256, 2059, 3105, 8464, 1476, 3152, 6488, 2330, 3529, 3293, 6409, 4966, 5274, 5269, 6415, 3008]
pd.options.display.float_format = "{:,.2f}".format

buy_prices_ror10 = np.array(buy_prices_ror10)
buy_prices_ror15 = np.array(buy_prices_ror15)
cur_prices = np.array(cur_prices).astype(float)

data = {
    "id": stock_ids,
    "公司": company_names,
    "種類": stock_types,
    "產業": industrys,
    "成交量": volumes,
    "買入價 (ror=10%)": buy_prices_ror10,
    "買入價 (ror=15%)": buy_prices_ror15,
    "賣出價": sell_prices,
    "現價": cur_prices,
    "距離出手價格": cur_prices - buy_prices_ror10,
}
df = pd.DataFrame(data) 

In [106]:
df

,id,公司,種類,產業,成交量,買入價 (ror=10%),買入價 (ror=15%),賣出價,現價,距離出手價格
0,2904,匯僑,上市,其他類股,50,18.92,14.47,21.97,24.00,5.08
1,3567,逸昌,上櫃,半導體類股,17,23.22,17.79,26.61,25.25,2.03
2,6216,居易,上市,通信網路類股,64,26.88,20.60,30.69,25.30,-1.58
3,4426,利勤,上市,紡織類股,57,26.82,19.47,47.20,27.20,0.38
4,2633,台灣高鐵,上市,航運類股,"3,311",16.24,12.47,18.27,30.50,14.26
...,...,...,...,...,...,...,...,...,...,...
65,4966,譜瑞-KY,上櫃,半導體類股,744,249.55,185.92,366.23,"1,130.00",880.45
66,5274,信驊,上櫃,半導體類股,141,251.52,192.56,290.23,"1,355.00","1,103.48"
67,5269,祥碩,上市,半導體類股,515,129.05,98.24,157.43,"1,545.00","1,415.95"
68,6415,矽力-KY,上市,半導體類股,157,238.64,174.22,404.81,"1,860.00","1,621.36"


In [107]:
df.to_csv("stock_analysis.csv", float_format='%.2f')

In [108]:
stock_should_buy_ror10 = df["id"][df["買入價 (ror=10%)"] >= df["現價"]]
stock_should_buy_ror15 = df["id"][df["買入價 (ror=15%)"] >= df["現價"]]
print ("stock_should_buy_ror10:", list(stock_should_buy_ror10))
print ("stock_should_buy_ror15:", list(stock_should_buy_ror15))

stock_should_buy_ror10: [6216, 6210, 9943, 6452, 1565]
stock_should_buy_ror15: [6452]
